# Covid-19 en Centroamérica
> Dashboard

- toc: false
- branch: master
- badges: true
- comments: false
- image: images/covid-19-ca.png
- permalink: /covid-19-ca/
- author: "Libre AI"
- categories: [centroamerica, covid19]

In [16]:
#hide
import math
import folium
import pandas as pd

from IPython.display import display, display_html, HTML, IFrame, Markdown

In [17]:
#hide_input

from datetime import datetime
utc_now = datetime.utcnow()

Markdown('> Last updated on {} UTC'.format(utc_now.strftime("%Y-%m-%d %H:%M")))

> Last updated on 2020-03-31 08:10 UTC

In [18]:
#hide

#latest case count
country_code2name = {'gt': 'Guatemala',
                     'bz': 'Belize',
                     'hn': 'Honduras',
                     'sv': 'El Salvador',
                     'ni': 'Nicaragua',
                     'cr': 'Costa Rica',
                     'pa': 'Panama'
                    }



In [19]:
#hide
confirmed = pd.read_csv('data/time_series_covid19_confirmed_global.csv')

In [20]:
#hide
def get_latest_count(df, country):
    df_country = df[df['Country/Region'] == country]
    # dates are columns, so we take the last column
    latest_count_date = df_country.columns[-1]
    latest_count = df_country[latest_count_date]
    return latest_count.values[0]

In [21]:
#hide 
cases_by_country = {c_code:get_latest_count(confirmed, country_code2name[c_code]) for c_code in country_code2name}
cases_by_country

{'gt': 36, 'bz': 3, 'hn': 139, 'sv': 30, 'ni': 4, 'cr': 330, 'pa': 989}

In [22]:
#hide

map_center_lat, map_center_long = (12.5, -87)
zoom = 6

In [23]:
#hide

class Country:
    def __init__(self, name, lat, long):
        super()
        self.cases = 0
        self.name = name
        self.lat = lat
        self.long = long


In [24]:
#hide

gt = Country(country_code2name['gt'], 14.63, -90.56)
bz = Country(country_code2name['bz'], 17.25, -88.80)
hn = Country(country_code2name['hn'], 14.08, -87.24)
sv = Country(country_code2name['sv'], 13.69, -89.25)
ni = Country(country_code2name['ni'], 12.10, -86.33)
cr = Country(country_code2name['cr'], 9.94, -84.15)
pa = Country(country_code2name['pa'], 9.08, -79.59)

countries = {'gt': gt, 'bz': bz, 'hn': hn, 'sv': sv, 'ni': ni, 'cr': cr, 'pa': pa}

In [25]:
#hide

# set latest cases
for c in countries:
    countries[c].cases = cases_by_country[c]

In [26]:
#hide

map_circles_html = []
for c in countries:
    c_circle = '''
        L.circle([{}, {}], {}*3000, {{
          color: 'red',
          fillColor: '#f03',
          fillOpacity: 0.5
          }}).addTo(mymap)
          .bindTooltip("<div><b>{}</b><br/><span style='color:red'><h3 style='display:inline'>{}</h3></span>&nbsp;personas infectadas</div>", {{sticky: true}});'''.format(countries[c].lat,
                                                                               countries[c].long,
                                                                               math.sqrt(countries[c].cases),
                                                                               countries[c].name,
                                                                               countries[c].cases)
    
    map_circles_html.append(c_circle)
    

In [50]:
#hide

m = folium.Map(location=[map_center_lat, map_center_long],
               zoom_start=zoom,
               max_zoom=zoom+1,
               min_zoom=zoom-1
              )

# tiles = 'cartodbdark_matter'
# tiles = 'cartodbpositron'
# tiles = 'stamentoner'
tiles = 'stamenterrain'

folium.TileLayer(tiles, min_zoom=zoom-1, max_zoom=zoom+1).add_to(m)

for c in countries:
    folium.CircleMarker(
        location=[countries[c].lat, countries[c].long],
        radius=math.sqrt(countries[c].cases),
        color='red',
        fill=True,
        fill_color='#f03',
        tooltip='''<div><b>{}</b><br/>
        <span style='color:red'><h3 style='display:inline'>{}</h3></span>
        &nbsp;personas infectadas</div>'''.format(countries[c].name, countries[c].cases)
    ).add_to(m)


m.save('../maps/covid19-ca-map.html')

In [51]:
# hide_input
IFrame('../maps/covid19-ca-map.html', width=800, height=600)

---
### Data Sources
- 2019 Novel Coronavirus COVID-19 (2019-nCoV) Data Repository by Johns Hopkins CSSE [https://github.com/CSSEGISandData/COVID-19]
